In [ ]:
# 
# .. _demo_pde_iterative_stokes_python_documentation:
# 
# Stokes equations with an iterative solver
# =========================================
# 
# This demo is implemented in a single Python file,
# :download:`demo_stokes-iterative.py`, which contains both the
# variational forms and the solver.
# 
# This demo illustrates how to:
# 
# * Read mesh and subdomains from file
# * Use mixed function spaces
# 
# 
# Equation and problem definition
# -------------------------------
# 
# Strong formulation
# ^^^^^^^^^^^^^^^^^^
# 
# .. math::
#         - \nabla \cdot (\nabla u + p I) &= f \quad {\rm in} \ \Omega, \\
#                         \nabla \cdot u &= 0 \quad {\rm in} \ \Omega. \\
# 
# .. note::
#         The sign of the pressure has been flipped from the classical
#         definition. This is done in order to have a symmetric (but not
#         positive-definite) system of equations rather than a
#         non-symmetric (but positive-definite) system of equations.
# 
# A typical set of boundary conditions on the boundary :math:`\partial
# \Omega = \Gamma_{D} \cup \Gamma_{N}` can be:
# 
# .. math::
#         u &= u_0 \quad {\rm on} \ \Gamma_{D}, \\
#         \nabla u \cdot n + p n &= g \,   \quad\;\; {\rm on} \ \Gamma_{N}. \\
# 
# 
# Weak formulation
# ^^^^^^^^^^^^^^^^
# 
# The Stokes equations can easily be formulated in a mixed variational
# form; that is, a form where the two variables, the velocity and the
# pressure, are approximated simultaneously. Using the abstract
# framework, we have the problem: find :math:`(u, p) \in W` such that
# 
# .. math::
#         a((u, p), (v, q)) = L((v, q))
# 
# for all :math:`(v, q) \in W`, where
# 
# .. math::
# 
#         a((u, p), (v, q))
#         &= \int_{\Omega} \nabla u \cdot \nabla v
#         - \nabla \cdot v \ p
#         + \nabla \cdot u \ q \, {\rm d} x, \\
#         L((v, q))
#         &= \int_{\Omega} f \cdot v \, {\rm d} x
#         + \int_{\partial \Omega_N} g \cdot v \, {\rm d} s. \\
# 
# The space :math:`W` should be a mixed (product) function space
# :math:`W = V \times Q`, such that :math:`u \in V` and :math:`q \in Q`.
# 
# 
# Domain and boundary conditions
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# 
# In this demo, we shall consider the following definitions of the input
# functions, the domain, and the boundaries:
# 
# * :math:`\Omega = [0,1]\times[0,1] \backslash {\rm dolphin}` (a unit
#   cube)
# * :math:`\Gamma_D =`
# * :math:`\Gamma_N =`
# * :math:`u_0 = (- \sin(\pi x_1), 0.0)` for :math:`x_0 = 1` and
#   :math:`u_0 = (0.0, 0.0)` otherwise
# * :math:`f = (0.0, 0.0)`
# * :math:`g = (0.0, 0.0)`
# 
# 
# Implementation
# --------------
# 
# The Stokes equations as formulated above result in a system of linear
# equations that is not positive-definite. Standard iterative linear
# solvers typically fail to converge for such systems. Some care must
# therefore be taken in preconditioning the systems of
# equations. Moreover, not all of the linear algebra backends support
# this. We therefore start by checking that either "PETSc" or "Tpetra"
# (from Trilinos) is available. We also try to pick MINRES Krylov
# subspace method which is suitable for symmetric indefinite problems.
# If not available, costly QMR method is choosen. ::

from dolfin import *
import matplotlib.pyplot as plt
parent_folder = "/home/shared"

# Test for PETSc or Tpetra
if not has_linear_algebra_backend("PETSc") and not has_linear_algebra_backend("Tpetra"):
    info("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
    exit()

if not has_krylov_solver_preconditioner("amg"):
    info("Sorry, this demo is only available when DOLFIN is compiled with AMG "
         "preconditioner, Hypre or ML.")
    exit()

if has_krylov_solver_method("minres"):
    krylov_method = "minres"
elif has_krylov_solver_method("tfqmr"):
    krylov_method = "tfqmr"
else:
    info("Default linear algebra backend was not compiled with MINRES or TFQMR "
         "Krylov subspace method. Terminating.")
    exit()

In [ ]:
import numpy as np
import pathlib
from smart.mesh_tools import gmsh_to_dolfin
from typing import Tuple, Union
from numpy.typing import NDArray
width = 10
height = 10
def sheet_with_sphere(
    sphereRad: float = 1,
    sheetWidth: float = width,
    sheetHeight: float = height,
    hOut: float = 0,
    hSphere: float = 0,
    interface_marker: int = 12,
    outer_marker: int = 10,
    inner_tag: int = 2,
    outer_tag: int = 1,
    half_cell: bool = True,
) -> Tuple[Mesh, MeshFunction, MeshFunction]:
    """
    Creates a 2D mesh representing a sphere inside of a larger domain

    Args:
        sphereRad: Radius of inner sphere
        sheetWidth: width of total 2d sheet
        sheetHeight: height of total 2d sheet
        hOut: mesh size outside the sphere
        hSphere: mesh size inside the sphere
        hInnerEdge: maximum mesh size at the edge
            of the inner compartment
        interface_marker: The value to mark facets on the interface with
        outer_marker: The value to mark facets on outer edge
        inner_tag: The value to mark the inside of the sphere
        outer_tag: The value to mark the region outside the sphere
        half_cell: If true, only use the right half of the geometry
    Returns:
        Tuple (mesh, facet_marker, cell_marker)
    """
    import gmsh

    if np.isclose(hSphere, 0):
        hSphere = 0.2 * sphereRad
    if np.isclose(hOut, 0):
        hOut = 0.1 * min([sheetWidth, sheetHeight])
    # Create the 2D mesh using gmsh
    gmsh.initialize()
    gmsh.model.add("spherePlus")
    # first add sheet centered at (0,0,0)
    if not half_cell:
        sheet = gmsh.model.occ.addRectangle(-sheetWidth/2, -sheetHeight/2, 0, sheetWidth, sheetHeight, 0)
        # Add inner_sphere (radius sphereRad, center (0,0,0))
        sphere = gmsh.model.occ.addDisk(0, 0, 0, sphereRad, sphereRad)
        # Create interface between shapes
        spherePlus, (sheet_map, sphere_map) = gmsh.model.occ.fragment(
            [(2, sheet)], [(2, sphere)]
        )
    else:
        sheet = gmsh.model.occ.addRectangle(0, -sheetHeight/2, 0, sheetWidth/2, sheetHeight)
        # Add inner_sphere (radius sphereRad, center (0,0,0))
        top_point = gmsh.model.occ.addPoint(0, sphereRad, 0)
        center_point = gmsh.model.occ.addPoint(0, 0, 0)
        bottom_point = gmsh.model.occ.addPoint(0, -sphereRad, 0)
        arc = gmsh.model.occ.addCircleArc(top_point, center_point, bottom_point, center=True)
        symm = gmsh.model.occ.addLine(top_point, bottom_point)
        sphere_bound = gmsh.model.occ.addCurveLoop([arc, symm])
        sphere = gmsh.model.occ.addPlaneSurface([sphere_bound])
        # Create interface between shapes
        spherePlus, (sheet_map, sphere_map) = gmsh.model.occ.fragment(
            [(2, sheet)], [(2, sphere)]
        )
    gmsh.model.occ.synchronize()

    # Get the outer boundary
    outer_bound = gmsh.model.getBoundary(spherePlus, oriented=False)
    outer_bound = [tag[1] for tag in outer_bound]
    # Get the inner boundary
    inner_bound = gmsh.model.getBoundary(sphere_map, oriented=False)
    inner_bound = [tag[1] for tag in inner_bound]
    # Add physical markers for facets (edges)
    gmsh.model.add_physical_group(1, outer_bound, tag=outer_marker)
    gmsh.model.add_physical_group(1, inner_bound, tag=interface_marker)

    # Physical markers for surfaces
    all_surfs = [tag[1] for tag in sheet_map]
    inner_surf = [tag[1] for tag in sphere_map]
    outer_surf = []
    for surf in all_surfs:
        if surf not in inner_surf:
            outer_surf.append(surf)
    gmsh.model.add_physical_group(2, outer_surf, tag=outer_tag)
    gmsh.model.add_physical_group(2, inner_surf, tag=inner_tag)

    def meshSizeCallback(dim, tag, x, y, z, lc):
        # inside sphere -> hSphere, outside -> hOut
        Rval = np.sqrt(x**2 + y**2)
        if Rval > sphereRad:
            lcTest = hOut
        else:
            lcTest = hSphere
        return lcTest

    gmsh.model.mesh.setSizeCallback(meshSizeCallback)
    # set off the other options for mesh size determination
    gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
    # this changes the algorithm from Frontal-Delaunay to Delaunay,
    # which may provide better results when there are larger gradients in mesh size
    gmsh.option.setNumber("Mesh.Algorithm", 5)

    gmsh.model.mesh.generate(2)
    tmp_folder = pathlib.Path(f"tmp_spherePlus")
    tmp_folder.mkdir(exist_ok=True)
    gmsh_file = tmp_folder / "spherePlus.msh"
    gmsh.write(str(gmsh_file))
    gmsh.finalize()

    # return dolfin mesh of max dimension (parent mesh) and marker functions mf_facet and mf_cell
    dmesh, mf1, mf_facet = gmsh_to_dolfin(str(gmsh_file), tmp_folder, 2)
    # remove tmp mesh and tmp folder
    gmsh_file.unlink(missing_ok=False)
    tmp_folder.rmdir()
    # return dolfin mesh, mf1 (1d tags) and mf_facet (2d tags)
    return (dmesh, mf1, mf_facet)

In [ ]:
mesh, mf1, mf_facet = sheet_with_sphere(hSphere=0.1, sphereRad=1.0)
File(f"{parent_folder}/mesh.pvd") << mesh
File(f"{parent_folder}/interface.pvd") << mf1


In [ ]:
# Next, we define the mesh (a UnitSquareMesh`) and a mixed finite element ``TH``.  Then
# we build a :py:class:`FunctionSpace
# <dolfin.functions.functionspace.FunctionSpace>` on this element.
# (This mixed finite element space is known as the Taylor--Hood elements
# and is a stable, standard element pair for the Stokes equations.) ::

# Create circle mesh in SMART
hmesh = [0.5,0.1,0.05]
hmeshO = [0.5,0.1,0.05]
L2 = []
L2O = []
Uv=1
sigma=1
a=1
meshrad = a**2
ur_sol=Expression("(Uv*sigma*x[0]*(x[1]))/((2*pow(a,2))*(1+sigma))",Uv=Uv,sigma=sigma,a=a, degree=2)
uz_sol=Expression("((Uv*sigma)/(2*(1+sigma)))*(1-(2*pow(x[0],2)+pow(x[1],2))/pow(a,2))",Uv=Uv,sigma=sigma,a=a,degree=2)
uvec_sol = Expression(("ur","uz","0"),ur=ur_sol,uz=uz_sol,degree=2)
uzo_sol = Expression("Uv*(0.75*a*pow(x[0],2)*pow((pow(x[0],2) + pow(x[1],2)),2.0)*(pow(a,2)*pow((pow(x[0],2) + pow(x[1],2)),1.5) - pow((pow(x[0],2) + pow(x[1],2)),2.5)*(0.666666666666667*sigma + 1)) - 2*pow((pow(x[0],2) + pow(x[1],2)),4.0)*(0.25*pow(a,3)*pow((pow(x[0],2) + pow(x[1],2)),0.5) - 0.75*a*pow((pow(x[0],2) + pow(x[1],2)),1.5)*(0.666666666666667*sigma + 1) + 0.5*pow((pow(x[0],2) + pow(x[1],2)),2.0)*(sigma + 1)))/(pow((pow(x[0],2) + pow(x[1],2)),6.0)*(sigma + 1))",Uv=Uv,sigma=sigma,a=a,degree=2)
uro_sol =  Expression("-0.75*Uv*a*x[0]*x[1]*(pow(a,2)*pow((pow(x[0],2) + pow(x[1],2)),1.5) - pow((pow(x[0],2) + pow(x[1],2)),2.5)*(0.666666666666667*sigma + 1))/(pow((pow(x[0],2) + pow(x[1],2)),4.0)*(sigma + 1))",sigma=sigma,Uv=Uv,a=a,degree=2)
uvecO_sol = Expression(("ur","uz","0"),ur=uro_sol,uz=uzo_sol,degree=2)

for i in range(0,len(hmesh)):
    mesh, mf_facet, mf_cell = sheet_with_sphere(sphereRad=a, hSphere=hmesh[i], interface_marker = 2, inner_tag = 1, outer_tag=10)
    mesh = create_meshview(mf_cell, 1)
    meshO, mf_facetO, mf_cellO = sheet_with_sphere(sphereRad=a, hSphere=hmeshO[i], interface_marker = 2, inner_tag = 1, outer_tag=10)
    mf_facet = MeshFunction("size_t", mesh, 1, 0)
    meshO = create_meshview(mf_cellO, 10)
    mf_facetO = MeshFunction("size_t", meshO, 1, 0)
    
    class AwayFromSymm(SubDomain):
        def inside(self, x, on_boundary):
            return x[0] > 0 and on_boundary
    away = AwayFromSymm()
    away.mark(mf_facet, 2) # mark all points off symm axis as 2

    class AwayFromSymmO(SubDomain):
        def inside(self, y, on_boundary):
            return y[0] > 0 and abs(y[1])<=a and y[0]<=a and on_boundary
    awayO = AwayFromSymmO()
    awayO.mark(mf_facetO, 3)
    
    class RightBound(SubDomain):
        def inside(self, y, on_boundary):
            return y[0]>a and abs(y[1])<height/2 and on_boundary
    awayO2 = RightBound()
    awayO2.mark(mf_facetO, 4)

    class ZBound(SubDomain):
        def inside(self, y, on_boundary):
            return y[0]<width/2 and y[0]>0 and abs(y[1])>a and on_boundary
    awayO3 = ZBound()
    awayO3.mark(mf_facetO, 5)

# Build function space
    P2 = VectorElement("Lagrange", mesh.ufl_cell(), 2)
    P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
    P1vec = VectorFunctionSpace(mesh, "P", 1)
    TH = P2 * P1
    P2O = VectorElement("Lagrange", meshO.ufl_cell(), 2)
    P1O = FiniteElement("Lagrange", meshO.ufl_cell(), 1)
    P1vecO = VectorFunctionSpace(meshO, "P", 1)
    THO = P2O * P1O
    W = FunctionSpace(mesh, TH)
    WO = FunctionSpace(meshO, THO)

    #Linear Combination BC:
    #def tanv_combine_expression(
    #x: NDArray[Union[np.float32, np.float64]],
    #) -> NDArray[Union[np.float32, np.float64, np.complex64, np.complex128]]:
     #   return np.stack(
    #        (
    #            np.sin(np.pi * np.sqrt(x[0] ** 2 + x[1] ** 2)),
    #            5 * x[1] * np.sin(np.pi * np.sqrt(x[0] ** 2 + x[1] ** 2)),
     #       )
     #   ).astype(default_scalar_type)


    #tanv_combine.interpolate(tanv_combine_expression)
    #tanv_combine.x.scatter_forward()

# Next, we define the boundary conditions:
    eta=10
    bc0 = DirichletBC(W.sub(0).sub(0), 0, mf_facet, 0)
    tanv = Expression(("(Uv*sigma*x[0]*(x[1]))/((2*pow(a,2))*(1+sigma))","((Uv*sigma)/(2*(1+sigma)))*(1-(2*pow(x[0],2)+pow((x[1]),2))/pow(a,2))","0.0"), Uv=Uv, sigma=sigma, a=a, degree=2)
    bc1 = DirichletBC(W.sub(0), tanv, mf_facet, 2)
    ds = Measure("ds",mesh,subdomain_data=mf_facet)
    dsO = Measure("ds",meshO,subdomain_data=mf_facetO)
    tanvO = Expression(("-0.75*Uv*a*x[0]*x[1]*(pow(a,2)*pow((pow(x[0],2) + pow(x[1],2)),1.5) - pow((pow(x[0],2) + pow(x[1],2)),2.5)*(0.666666666666667*sigma + 1))/(pow((pow(x[0],2) + pow(x[1],2)),4.0)*(sigma + 1))","Uv*(0.75*a*pow(x[0],2)*pow((pow(x[0],2) + pow(x[1],2)),2.0)*(pow(a,2)*pow((pow(x[0],2) + pow(x[1],2)),1.5) - pow((pow(x[0],2) + pow(x[1],2)),2.5)*(0.666666666666667*sigma + 1)) - 2*pow((pow(x[0],2) + pow(x[1],2)),4.0)*(0.25*pow(a,3)*pow((pow(x[0],2) + pow(x[1],2)),0.5) - 0.75*a*pow((pow(x[0],2) + pow(x[1],2)),1.5)*(0.666666666666667*sigma + 1) + 0.5*pow((pow(x[0],2) + pow(x[1],2)),2.0)*(sigma + 1)))/(pow((pow(x[0],2) + pow(x[1],2)),6.0)*(sigma + 1))","0"),sigma=sigma,Uv=Uv,a=a,degree=2)
    bc2 = DirichletBC(WO.sub(0), tanvO, mf_facetO, 3)
    bc3 = DirichletBC(WO.sub(0), (0,-Uv,0), mf_facetO, 4)
    bc4 = DirichletBC(WO.sub(0), (0,-Uv,0), mf_facetO, 5)



    x = SpatialCoordinate(mesh)
    y = SpatialCoordinate(meshO)
    # define axisymmetric gradient and divergence
    # see, for instance, formulas here (en.wikipedia.org/wiki/Del_in_cylindrical_and_spherical_coordinates)
    def axisymm_grad(v): # axis of symm is x[0]=0
    # in addition to this definition, x[0] is included in all integrals
        return sym(as_tensor([[v[0].dx(0), 0, v[0].dx(1)],
                            [0, v[0]/x[0], 0],
                            [v[1].dx(0), 0, v[2].dx(1)]]))
    def axisymm_gradO(vO): # axis of symm is x[0]=0
    # in addition to this definition, x[0] is included in all integrals
        return sym(as_tensor([[vO[0].dx(0), 0, vO[0].dx(1)],
                            [0, vO[0]/y[0], 0],
                            [vO[1].dx(0), 0, vO[2].dx(1)]]))

    def axisymm_div(v): # axis of symm is x[0]=0
        # in addition to this definition, x[0] is included in all integrals
        return v[0].dx(0) + v[0]/x[0] + v[1].dx(1)
    def axisymm_divO(vO): # axis of symm is x[0]=0
        # in addition to this definition, x[0] is included in all integrals
        return vO[0].dx(0) + vO[0]/y[0] + vO[1].dx(1)

# Collect boundary conditions
    bcs = [bc1]
    bcsO = [bc3,bc4]

# The bilinear and linear forms corresponding to the weak mixed
# formulation of the Stokes equations are defined as follows: ::

# Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    (uO, pO) = TrialFunctions(WO)
    (vO, qO) = TestFunctions(WO)
    f = Constant((0.0, 0.0, 0.0))
    ax = inner(axisymm_grad(u), axisymm_grad(v))*x[0]*dx + axisymm_div(v)*p*x[0]*dx + q*axisymm_div(u)*x[0]*dx
    axO = inner(axisymm_gradO(uO), axisymm_gradO(vO))*y[0]*dx + axisymm_divO(vO)*pO*y[0]*dx + qO*axisymm_divO(uO)*y[0]*dx + ((eta*vO[0]*(uO[0]*y[1]-uO[1]*y[0]-a*(y[0]*y[1]*((uO[0].dx(0))-(uO[1].dx(1)))-y[0]**2*(uO[1].dx(0))+y[1]**2*(uO[0].dx(1)))/sqrt(y[0]**2+y[1]**2))))*dsO(3)
    L = inner(f, v)*x[0]*dx
    LO = inner(f, vO)*y[0]*dx


# We can now use the same :py:class:`TrialFunctions
# <dolfin.functions.function.TrialFunction>` and
# :py:class:`TestFunctions <dolfin.functions.function.TestFunction>` to
# define the preconditioner matrix. We first define the form
# corresponding to the expression for the preconditioner (given in the
# initial description above): ::

# Form for use in constructing preconditioner matrix
    b = inner(axisymm_grad(u), axisymm_grad(v))*x[0]*dx + p*q*x[0]*dx
    bO = inner(axisymm_gradO(uO), axisymm_gradO(vO))*y[0]*dx + pO*qO*y[0]*dx

# Next, we want to assemble the matrix corresponding to the bilinear
# form and the vector corresponding to the linear form of the Stokes
# equations. Moreover, we want to apply the specified boundary
# conditions to the linear system. However, :py:func:`assembling
# <dolfin.fem.assembling.assemble>` the matrix and vector and applying a
# :py:func:`DirichletBC <dolfin.fem.bcs.DirichletBC>` separately will
# possibly result in a non-symmetric system of equations. Instead, we
# can use the :py:func:`assemble_system
# <dolfin.fem.assembling.assemble_system>` function to assemble both the
# matrix ``A``, the vector ``bb``, and apply the boundary conditions
# ``bcs`` in a symmetric fashion: ::

# Assemble system
    A, bb = assemble_system(ax, L, bcs)
    AO, bbO = assemble_system(axO, LO, bcsO)

# We do the same for the preconditioner matrix ``P`` using the linear
# form ``L`` as a dummy form: ::

# Assemble preconditioner system
    P, btmp = assemble_system(b, L, bcs)
    PO, btmpO = assemble_system(bO, LO, bcsO)

# Next, we specify the iterative solver we want to use, in this case a
# :py:class:`KrylovSolver <dolfin.cpp.KrylovSolver>`. We associate the
# left-hand side matrix ``A`` and the preconditioner matrix ``P`` with
# the solver by calling :py:func:`solver.set_operators
# <dolfin.cpp.GenericLinearSolver.set_operators>`. ::

    # Create Krylov solver and AMG preconditioner
    solver = KrylovSolver(krylov_method, "amg")

    # Associate operator (A) and preconditioner matrix (P)
    solver.set_operators(A, P)
    U = Function(W)
    solver.solve(U.vector(), bb)

# We are now almost ready to solve the linear system of equations. It
# remains to specify a :py:class:`Vector <dolfin.cpp.Vector>` for
# storing the result. For easy manipulation later, we can define a
# :py:class:`Function <dolfin.functions.function.Function>` and use the
# vector associated with this Function. The call to
# :py:func:`solver.solve <dolfin.cpp.KrylovSolver.solve>` then looks as
# follows ::

# Solve
    solverO = KrylovSolver(krylov_method, "amg")
    solverO.set_operators(AO, PO)
    UO = Function(WO) 
    solverO.solve(UO.vector(), bbO)


# Get sub-functions
    u, p = U.split()
    uO, pO = UO.split()
    L2.append(sqrt(assemble(((u.sub(0) - ur_sol)**2 + (u.sub(1) - uz_sol)**2)*x[0]*dx)/assemble(x[0]*dx)))
    L2O.append(sqrt(assemble(((uO.sub(0) - uro_sol)**2 + (uO.sub(1) - uzo_sol)**2)*y[0]*dx)/assemble(y[0]*dx)))

    ufile_pvd = File(f"{parent_folder}/velocity.pvd")
    ufile_pvd << u
    ufile_pvd = File(f"{parent_folder}/velocityO.pvd")
    ufile_pvd << uO
    File(f"{parent_folder}/uvec.pvd") << project(uvec_sol, P1vec)
    File(f"{parent_folder}/uvecO.pvd") << project(uvecO_sol, P1vecO)
    
plt.loglog(hmesh,L2)

In [ ]:
plt.loglog(hmeshO,L2O)

In [ ]:
File("mf_facet.pvd") << mf_facet